<a href="https://colab.research.google.com/github/bttrung/secure-private-ai-scholarship/blob/master/8__Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install syft

In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [5]:
y

tensor([ 2,  4,  6,  8, 10])

In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [9]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [11]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [14]:
bob._objects

{17834038510: tensor([1, 2, 3, 4, 5])}

In [15]:
x.location

<VirtualWorker id:bob #objects:1>

In [16]:
x.id_at_location

17834038510

In [17]:
x.id

55106152970

In [18]:
x.owner

<VirtualWorker id:me #objects:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [20]:
x

(Wrapper)>[PointerTensor | me:55106152970 -> bob:17834038510]

In [21]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

In [0]:
# Playing with Remote Tensors
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [26]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [29]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [0]:
# Introducing Remote Arithmetic
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [31]:
x

(Wrapper)>[PointerTensor | me:47847172715 -> bob:66365770439]

In [32]:
y

(Wrapper)>[PointerTensor | me:55021208759 -> bob:86033625828]

In [0]:
z = x + y

In [34]:
z

(Wrapper)>[PointerTensor | me:72278100357 -> bob:18633844136]

In [35]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [36]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:78480776260 -> bob:24785541585]

In [37]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [40]:
z.backward()

(Wrapper)>[PointerTensor | me:94533220802 -> bob:12205793521]

In [41]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [42]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
# Learn a Simple Linear Model
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)


In [45]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)


tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


In [0]:
bob = bob.clear_objects()

In [47]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [49]:
bob._objects

{6334335223: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [51]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{50902737765: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "aaavvvv"

In [55]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
x

(Wrapper)>[PointerTensor | me:28076720630 -> bob:62335097365]

In [58]:
bob._objects

{62335097365: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "aaavvvv"

In [60]:
bob._objects

{62335097365: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [62]:
bob._objects

{62335097365: tensor([1, 2, 3, 4, 5])}

In [63]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [65]:
bob._objects

{54211820596: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [67]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [69]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
# Toy Federated Learning
from torch import nn, optim
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [72]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()


tensor(1.5174)
tensor(0.1562)
tensor(0.0420)
tensor(0.0251)
tensor(0.0179)
tensor(0.0132)
tensor(0.0097)
tensor(0.0072)
tensor(0.0054)
tensor(0.0040)
tensor(0.0030)
tensor(0.0023)
tensor(0.0017)
tensor(0.0013)
tensor(0.0010)
tensor(0.0007)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)


In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())


In [77]:
train()

tensor(7.4272, requires_grad=True)
tensor(0.9993, requires_grad=True)
tensor(0.5822, requires_grad=True)
tensor(0.7557, requires_grad=True)
tensor(0.2684, requires_grad=True)
tensor(0.4704, requires_grad=True)
tensor(0.1586, requires_grad=True)
tensor(0.2917, requires_grad=True)
tensor(0.0965, requires_grad=True)
tensor(0.1825, requires_grad=True)
tensor(0.0597, requires_grad=True)
tensor(0.1154, requires_grad=True)
tensor(0.0375, requires_grad=True)
tensor(0.0738, requires_grad=True)
tensor(0.0240, requires_grad=True)
tensor(0.0478, requires_grad=True)
tensor(0.0157, requires_grad=True)
tensor(0.0314, requires_grad=True)
tensor(0.0104, requires_grad=True)
tensor(0.0208, requires_grad=True)
tensor(0.0071, requires_grad=True)
tensor(0.0140, requires_grad=True)
tensor(0.0049, requires_grad=True)
tensor(0.0095, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0018, requi

In [78]:
# Advanced Remote Execution Tools
bob.clear_objects()
alice.clear_objects()


<VirtualWorker id:alice #objects:0>

In [79]:
x = th.tensor([1,2,3,4,5]).send(bob)
x = x.send(alice)
bob._objects

{23555972408: tensor([1, 2, 3, 4, 5])}

In [80]:
alice._objects

{56148913635: (Wrapper)>[PointerTensor | alice:56148913635 -> bob:23555972408]}

In [81]:
y = x + x
y

(Wrapper)>[PointerTensor | me:1003729808 -> alice:72779108050]

In [82]:
bob._objects

{23555972408: tensor([1, 2, 3, 4, 5]),
 33394813181: tensor([ 2,  4,  6,  8, 10])}

In [83]:
alice._objects

{56148913635: (Wrapper)>[PointerTensor | alice:56148913635 -> bob:23555972408],
 72779108050: (Wrapper)>[PointerTensor | alice:72779108050 -> bob:33394813181]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [86]:
bob._objects

{94068723783: tensor([1, 2, 3, 4, 5])}

In [87]:
alice._objects

{59795825826: (Wrapper)>[PointerTensor | alice:59795825826 -> bob:94068723783]}

In [88]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:59795825826 -> bob:94068723783]

In [89]:
bob._objects

{94068723783: tensor([1, 2, 3, 4, 5])}

In [90]:
alice._objects

{}

In [91]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [92]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [94]:
bob._objects

{67767962238: tensor([1, 2, 3, 4, 5])}

In [95]:
alice._objects

{25875710907: (Wrapper)>[PointerTensor | alice:25875710907 -> bob:67767962238]}

In [0]:
del x

In [97]:
bob._objects

{}

In [98]:
alice._objects

{}

In [99]:
# Pointer Chain Operations
bob.clear_objects()
alice.clear_objects()


<VirtualWorker id:alice #objects:0>

In [100]:
x = th.tensor([1,2,3,4,5]).send(bob)
bob._objects

{72009092358: tensor([1, 2, 3, 4, 5])}

In [101]:
alice._objects

{}

In [102]:
x.move(alice)

(Wrapper)>[PointerTensor | me:40450982586 -> alice:82002013169]

In [103]:
bob._objects

{}

In [104]:
alice._objects

{82002013169: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [106]:
bob._objects

{75669635836: tensor([1, 2, 3, 4, 5])}

In [107]:
alice._objects

{82002013169: tensor([1, 2, 3, 4, 5]),
 93861239448: (Wrapper)>[PointerTensor | alice:93861239448 -> bob:75669635836]}

In [108]:
x.remote_get()

(Wrapper)>[PointerTensor | me:27204890722 -> alice:93861239448]

In [109]:
bob._objects

{}

In [110]:
alice._objects

{82002013169: tensor([1, 2, 3, 4, 5]), 93861239448: tensor([1, 2, 3, 4, 5])}

In [111]:
x.move(bob)

(Wrapper)>[PointerTensor | me:98781119301 -> bob:27204890722]

In [113]:
bob._objects

{27204890722: tensor([1, 2, 3, 4, 5])}

In [114]:
alice._objects

{82002013169: tensor([1, 2, 3, 4, 5])}